<a href="https://colab.research.google.com/github/angelozero/machine_learning/blob/main/employe_attrition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook: Análise da Satisfação dos Funcionários

## Introdução
Neste projeto, nosso objetivo é analisar a satisfação dos funcionários em uma empresa, utilizando um conjunto de dados que contém informações sobre desempenho, horas trabalhadas, e outros fatores. O objetivo é prever a satisfação dos funcionários com base em suas características.

O conjunto de dados contém informações como identificação dos funcionários, níveis de satisfação, avaliações de desempenho, envolvimento em projetos, entre outros. Essas informações nos permitirão entender os fatores que impactam a satisfação dos funcionários.


## Carga de Dados

### Importação de bibliotecas necessárias

In [45]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

## Visualização Básica dos Dados

In [36]:
# Carregando o dataset
df = pd.read_csv('/content/Employee Attrition.csv')

# Visualização básica dos dados
df.head(10)

,Emp ID,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years,dept,salary
0,1.0,0.38,0.53,2.0,157.0,3.0,0.0,0.0,sales,low
1,2.0,0.80,0.86,5.0,262.0,6.0,0.0,0.0,sales,medium
2,3.0,0.11,0.88,7.0,272.0,4.0,0.0,0.0,sales,medium
3,4.0,0.72,0.87,5.0,223.0,5.0,0.0,0.0,sales,low
4,5.0,0.37,0.52,2.0,159.0,3.0,0.0,0.0,sales,low
5,6.0,0.41,0.50,2.0,153.0,3.0,0.0,0.0,sales,low
6,7.0,0.10,0.77,6.0,247.0,4.0,0.0,0.0,sales,low
7,8.0,0.92,0.85,5.0,259.0,5.0,0.0,0.0,sales,low
8,9.0,0.89,1.00,5.0,224.0,5.0,0.0,0.0,sales,low
9,10.0,0.42,0.53,2.0,142.0,3.0,0.0,0.0,sales,low


## Pré-processamento dos Dados
### Limpeza de dados

In [37]:
# Verificando valores ausentes
df.isnull().sum()

# Tratamento de valores ausentes (se necessário)
df.dropna(inplace=True)

# Remoção de duplicados
df.drop_duplicates(inplace=True)

## Separação entre variáveis independentes e dependentes

In [38]:
# Supondo que a variável dependente seja 'Satisfaction'
X = df.drop('satisfaction_level', axis=1)

## Codificação de variáveis categóricas

In [39]:
# Converte variáveis categóricas em variáveis dummy
X_encoded = pd.get_dummies(X, drop_first=True)

# Converte para categórica
y = (df['satisfaction_level'] >= 0.5).astype(int)

## Divisão dos dados em conjuntos de treino e teste (holdout)

In [40]:
# Divisão dos dados em conjuntos de treino e teste (holdout)
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

## Transformação de Dados
### Normalização e padronização dos dados

In [41]:
# Normalização e Padronização dos Dados
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Modelagem
### Criação de Pipelines para Cada Modelo

In [42]:
# Criação de Pipelines para Cada Modelo
models = {
    'KNN': Pipeline([('scaler', StandardScaler()), ('knn', KNeighborsClassifier())]),
    'Decision Tree': Pipeline([('scaler', StandardScaler()), ('tree', DecisionTreeClassifier())]),
    'Naive Bayes': Pipeline([('scaler', StandardScaler()), ('nb', GaussianNB())]),
    'SVM': Pipeline([('scaler', StandardScaler()), ('svm', SVC())])
}

results = {}
for model_name, model in models.items():
    model.fit(X_train_scaled, y_train)  # Treinamento do modelo
    y_pred = model.predict(X_test_scaled)  # Predição
    results[model_name] = accuracy_score(y_test, y_pred)  # Avaliação

## Otimização de Hiperparâmetros
### Uso de GridSearchCV

In [43]:
# Definindo o grid de hiperparâmetros
param_grid = {
    'KNN': {'knn__n_neighbors': [3, 5, 7]},
    'Decision Tree': {'tree__max_depth': [None, 10, 20]},
    'Naive Bayes': {},  # Naive Bayes não tem hiperparâmetros relevantes para ajuste
    'SVM': {'svm__C': [0.1, 1, 10]}
}

best_models = {}
for model_name, model in models.items():
    # Verifica se há hiperparâmetros a serem ajustados
    if param_grid[model_name]:
        grid = GridSearchCV(model, param_grid[model_name], cv=5)
        grid.fit(X_train_scaled, y_train)
        best_models[model_name] = grid.best_estimator_
    else:
        # Treina o modelo sem ajuste de hiperparâmetros
        model.fit(X_train_scaled, y_train)
        best_models[model_name] = model

## Avaliação dos Modelos
### Comparação de Resultados

In [49]:
for model_name, model in best_models.items():
    y_pred = model.predict(X_test_scaled)
    print(f"Model: {model_name}")
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Classification Report:\n", classification_report(y_test, y_pred))
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
    print("\n")

Model: KNN
Accuracy: 0.8376666666666667
Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.59      0.69       911
           1       0.84      0.95      0.89      2089

    accuracy                           0.84      3000
   macro avg       0.83      0.77      0.79      3000
weighted avg       0.84      0.84      0.83      3000

Confusion Matrix:
 [[ 534  377]
 [ 110 1979]]


Model: Decision Tree
Accuracy: 0.8503333333333334
Classification Report:
               precision    recall  f1-score   support

           0       0.80      0.67      0.73       911
           1       0.87      0.93      0.90      2089

    accuracy                           0.85      3000
   macro avg       0.83      0.80      0.81      3000
weighted avg       0.85      0.85      0.85      3000

Confusion Matrix:
 [[ 614  297]
 [ 152 1937]]


Model: Naive Bayes
Accuracy: 0.7963333333333333
Classification Report:
               precision    recall  f1-sc

## Exportação do Modelo
### Salvamento do Melhor Modelo

In [48]:
import joblib

# Salvar o melhor modelo
joblib.dump(best_models['SVM'], 'best_model_employee_attrition.pkl')

['best_model_employee_attrition.pkl']